物件導向程式
===

In [4]:
# 物件導向的最基礎，我們定義class，然後用它宣告我們的物件/實體(instance)
# class 可以定義attribute(成員)、method(方法)，還有一些既定方法像是初始化__init__
# class裡的method，跟function基本上是一樣的，但第一個參數是self，會指向此實體本身

# 定義一個class叫做Person
# 我們習慣把class的名稱定義成UpperCaseCamelCase，字的開頭大寫
# 如果有多個字組成則在每個字的開頭大寫連起來
class Person:
  
  # 定義class attribute，所有實體會共用它
  kind = 'Person'
  
  # 初始化，當宣告實體的時候這個方法會被呼叫
  # 用self.xxx來定義實體的 attribute，每個被宣告的實體會有自己的attribute 
  def __init__(self, name, age):
    self.name = name; 
    self.age = age;
  
  # 定義實體的方法
  def say_hi(self):
    print(
      f'Hi, my name is {self.name}, I am {self.age} years old'
    )

if __name__ == '__main__':
    # 使用定義好的Class宣導實體(instance)
    mike = Person('Mike', 5) # 帶進去的參數Mike與5分辨對應__init__裡的name與age
    john = Person('John', 10)

    # 呼叫實體的方法
    mike.say_hi() # output: Hi, my name is Mike, I am 5 years old
    john.say_hi() # output: Hi, my name is John, I am 10 years old

    # 印出實體attribute
    print(mike.name) # output: Mike
    print(john.name) # output: John

    # class attribute是共用的，不管是class本身或是instance都能使用
    print(mike.kind) # Person
    print(Person.kind) # Person

Hi, my name is Mike, I am 5 years old
Hi, my name is John, I am 10 years old
Mike
John
Person
Person


In [99]:
from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import csv
import time
import pandas as pd
import numpy as np
import pyodbc
import logging

In [100]:
# 連線SQL
DBdata = {'ip':'192.168.99.142', 'db':'CMAPP'}
user142 = 'cmapp'
pwd142 = '0000'
conn_DBdata = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DBdata['ip'] + \
              ';DATABASE=' + DBdata['db'] + ';UID=' + user142 + ';PWD=' + pwd142)
DBdata_cr = conn_DBdata.cursor()
TableName = 'FB_AdsData'

In [103]:
# 提取流水號、報告日期
def Query_Both_SN_RD(TableName):
    query_both = """
        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[{}]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[{}])
        """
    print(query_both.format(TableName, TableName))
    DBdata_cr.execute(query_both.format(TableName, TableName))
    nums = DBdata_cr.fetchall()
    conn_DBdata.commit()
    return nums


# 清洗時間(Reporting_time)欄位，並更新
def Clean_both_SN_RD(uncleaned_data):
    index = 0
    list_of_data = [k for k in str(uncleaned_data).strip('[]').strip('()').split(',')]
    for i in list_of_data:
#         print(i.strip().strip('Decimal').strip('()').strip("'"), '\n')
        cleaned_data = i.strip().strip('Decimal').strip('()').strip("'")
        
        # 將清洗好的資料儲存
        if index%2 == 0:
            index += 1
            try:
                cleaned_SN = int(float(cleaned_data))
            except ValueError as e_reporting_date:
                print(e_reporting_date)
            
        elif index%2 == 1: 
            index += 1
            try:
                cleaned_RD = int(float(cleaned_data))
            except ValueError as e_reporting_date:
                print(e_reporting_date)
    return cleaned_SN, cleaned_RD


# 將Reporting_Date回溯1個月前
def RD_date_back(cleaned_RD):
    # 將str變成時間格式(datetime)，才能做計算
    reporting_date = datetime.strptime(str(cleaned_RD), '%Y%m%d')
    # 將日期回溯1個月
    reporting_date = reporting_date.date() - relativedelta(months = 1)
    return reporting_date


# 增加流水號 & 廣告類別
def Expand_df(df): 
    # 第幾排插入流水號 
    idx_SN = 0
    idx_CA = 11
    Serial_Number = []
    # 廣告類別
    Category = 'FB'

    # 增加Serial_Number & Catergory(FB, GOOGL, AAPL)
    for order in range(len(df)):
        s_num = str(Reporting_Date[order]) + '000' + str(int(str(cleaned_SN)[8:]) + order + 1)
        Serial_Number.append(int(s_num))
    
    # 將資料插入
    df.insert(loc = idx_SN, column = 'Serial_Number', value = Serial_Number)
    df.insert(loc = idx_CA, column = 'Category', value = Category)
    return df


# 將CSV內檔案的
def None_to_Null(df):
    for col in df.columns:
        df[col] = df[col].apply(lambda x: None if x == '' else x)
    return df 


# 將df的資料備份下來
def Df_to_Csv(df):
    # 將df內的Data儲存成CSV file
    df.to_csv('{}_FBAdsTest.csv'.format(datetime.now().strftime('%Y%m%d')), index = False, encoding='utf_8_sig')
    return None


# 刪除資料
def DelSQL(TableName, start_time, now_time):
    str_query = """
        DELETE
        FROM [CMAPP].[dbo].[{}]
        WHERE Reporting_Date BETWEEN {} AND {}
        """
    print(str_query.format(TableName, start_time, now_time))
    DBdata_cr.execute(str_query.format(TableName, start_time, now_time))
    conn_DBdata.commit()   


# 寫入資料
def InsertSQL(df, TableName):
    str_query = """
        INSERT INTO [CMAPP].[dbo].[{}]([Serial_Number],[Campaign_Name],[Adset_Name],[Objective],[Amount_Spent],[Amount_Install],\
        [Amount_Purchase],[Purchase_Conversion_Value],[Amount_Subscribe],[Subscribe_Conversion_Value],[Reporting_Date]\
        ,[Category])VALUES({})
        """
    # 將csv DataFrame每筆資料變成list形式
    insert_data = df.values.tolist()
    for i in range(len(insert_data)):
#         print(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))      
        DBdata_cr.execute(str_query.format(TableName, str(insert_data[i]).strip('[]').replace('None', 'null')))
        conn_DBdata.commit() 

## 執行所有程式的地方

In [101]:
uncleaned_data = Query_Both_SN_RD(TableName)
print(uncleaned_data)


        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[FB_AdsData]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[FB_AdsData])
        
[(Decimal('2021010500034726'), '20210105')]


In [104]:
cleaned_SN, cleaned_RD = Clean_both_SN_RD(uncleaned_data)
print(cleaned_SN, cleaned_RD)

2021010500034726 20210105


In [106]:
# 回朔時間
new_reporting_date = RD_date_back(cleaned_RD)
print('new_reporting_date(回溯一個月的時間): ', new_reporting_date.strftime('%Y%m%d'), end ='\n')

# 開始爬蟲的時間
start_time = new_reporting_date 
print('start_time(開始爬蟲時間): ', start_time.strftime('%Y%m%d'))

# 結束爬蟲的時間
now_time = datetime.now()
print('now_time(結束爬蟲時間):  ', now_time.strftime('%Y%m%d'))
end_time = new_reporting_date + relativedelta(weeks = 1)
print('end_time(給主程式跑的一星期): ', end_time.strftime('%Y%m%d'))

new_reporting_date(回溯一個月的時間):  20201205
start_time(開始爬蟲時間):  20201206
now_time(結束爬蟲時間):   20210108
end_time(給主程式跑的一星期):  20201212


In [107]:
# 刪除資料
DelSQL(TableName, start_time.strftime('%Y%m%d'), now_time.strftime('%Y%m%d'))


        DELETE
        FROM [CMAPP].[dbo].[FB_AdsData]
        WHERE Reporting_Date BETWEEN 20201206 AND 20210108
        


In [108]:
# 拿取dataFRame內的資料
uncleaned_data = Query_Both_SN_RD(TableName)
print(uncleaned_data)


        SELECT [Serial_Number], [Reporting_Date]
        FROM [CMAPP].[dbo].[FB_AdsData]
        WHERE  [Serial_Number] = (SELECT MAX(Serial_Number) 
        FROM [CMAPP].[dbo].[FB_AdsData])
        
[(Decimal('2020120500030800'), '20201205')]


In [110]:
cleaned_SN, cleaned_RD = Clean_both_SN_RD(uncleaned_data)
print(cleaned_SN, cleaned_RD)

2020120500030800 20201205


In [111]:
# 起始值
"""
access_token: 廣告金鑰
ad_account_id: 廣告帳戶
data_back: 往前爬多久的資料(幾個月)
fields: FB_Ads行銷廣告資料
count: 計算整體while迴圈跑幾次
stop_flag: 當結束時間已經超過現在時間，使迴圈再跑一次，然後停止迴圈
"""
access_token = 'EAAU5W2nIXuUBAIjps8JaNyhVzjh5mCywvWJDW6tleQXMGEl2xJMdlrfWDZB4wxZCRUU0Ed0FAKXT2acm31ZATt39m3u62ash9SP1La5BHgQOtEIBhCa5prRVDOuzZCPHAWeGPjBWCeZCegS3cmsu8ZBsfvURWFibQByAgGVqC53OHkxZA0pxixoCaXp1fKkLQgZD'
ad_account_id = 'act_132799196821088'
count = 0
stop_flag = 0
FacebookAdsApi.init(access_token = access_token)

# Insight API中的fields資料(控制需要的欄位資料)
fields = [
    'spend',
    'campaign_name',
    'adset_name',
    'campaign_id',
    'actions',
    'objective',
    'action_values',  
    'conversions',
    'conversion_values',
]

# 起始資料放置位置
Campaign_Name = list()
Adset_Name = list()
Objective = list()
Amount_Spent = list()
Amount_Install = list()
Amount_Purchase = list()
Purchase_Conversion_Value = list()
Amount_Subscribe = list()
Subscribe_Conversion_Value = list()
Reporting_Date = list()


# 紀錄程式跑多久時間
run_time = time.time()

# 起始時間、結束時間
start_time, end_time = start_time, end_time
# 現在時間
now = datetime.now().date()


# 比較時間前後順序
while True:
    # 計算迴圈跑幾次
    count += 1
    
    # Insight API中的fields資料(控制日期條件、資料的層級)
    # start_time和end_time需要為string(%Y-%m-%d)
    params = {
        'time_range': {
                    'since': str(start_time),\
                    'until': str(end_time),
        }, 
        # level從campaign改成adset
        'level': {'adset'},
        'time_increment': '1',
        # 手動設定的歸因: 7天點擊後(和廣告預設值有落差)
        'action_attribution_windows': {'7d_click'},
    }
    
    try:
        # 廣告帳號獲取廣告Insights資訊
        acc_insights = AdAccount(ad_account_id).get_insights(
            fields = fields,
            params = params,
        )
    except FacebookRequestError as error:
        print(error)
    
    # 了解時間怎麼走
    print(start_time.strftime('%Y-%m-%d'))
    print(end_time.strftime('%Y-%m-%d'))
    
    if stop_flag == 0:
    # 起始時間(start_time)和結束時間(end_time)，一次取一星期
        start_time = end_time + relativedelta(days = 1)
    else:
        start_time = end_time
        
    end_time = end_time + relativedelta(weeks = 1)
    
    # 從廣告Inishgts資訊中，清洗、篩選資料
    for acc_insight in acc_insights:     
        campaign_name = ''
        adset_name = ''
        amount_spent = ''
        objective = ''
        amount_subscribe = '' 
        subscribe_conversion_value = ''
        amount_install = '' 
        amount_purchase = ''    
        purchase_conversion_value = ''
        reporting_date = ''           
                                  
        # insights/campaign_name得到 "產品包名稱"
        if 'campaign_name' in acc_insight:
            campaign_name = acc_insight['campaign_name']
            if '⛔️股市爆料同學會' in campaign_name:
                campaign_name = campaign_name.replace('⛔️', '')
                
        # insights/adset_name得到 "產品名稱"
        if 'adset_name' in acc_insight:
            adset_name = acc_insight['adset_name']
                
        # insights/spend得到 "總支出成本"
        if 'spend' in acc_insight:    
            amount_spent = acc_insight['spend']

        # insights/objective得到 "廣告目標"
        if 'objective' in acc_insight:
            objective = acc_insight['objective']

        # insights/date_stop得到 "報表終止日期"
        if 'date_stop' in acc_insight:
            reporting_date = datetime.strptime(acc_insight['date_stop'], '%Y-%m-%d').strftime('%Y%m%d')

        # insights/actions得到 "軟體下載數/課程購買數"
        if 'actions' in acc_insight:
            for action in acc_insight['actions']:
                if action['action_type'] == 'mobile_app_install': 
                    amount_install = action['value']
                if action['action_type'] == 'purchase':
                    amount_purchase = action['value']

        # insights/action_values得到 "購買轉換價值"
        if 'action_values' in acc_insight:
            for ac_values in acc_insight['action_values']:
                if ac_values['action_type'] == 'omni_purchase':
                    purchase_conversion_value = ac_values['value']

        # insights/conversions得到 "訂閱數"
        if 'conversions' in acc_insight:
            for cv in acc_insight['conversions']:
                if cv['action_type'] == 'subscribe_total':
                    try:
                        amount_subscribe = cv['7d_click']
                    except:
                        amount_subscribe = cv['value']

        # insights/conversion_values得到 "訂閱轉換價值"             
        if 'conversion_values' in acc_insight:
            for cv_values in acc_insight['conversion_values']:
                if cv_values['action_type'] == 'subscribe_total':
                    try:
                        subscribe_conversion_value = cv_values['7d_click']
                    except:
                        subscribe_conversion_value = cv_values['value']
                        
        # 將資料以List的形式儲存
        try:
            Campaign_Name.append(campaign_name)
            Adset_Name.append(adset_name)
            Objective.append(objective)
            Amount_Spent.append(amount_spent)
            Amount_Install.append(amount_install)
            Amount_Purchase.append(amount_purchase)
            Purchase_Conversion_Value.append(purchase_conversion_value)
            Amount_Subscribe.append(amount_subscribe)
            Subscribe_Conversion_Value.append(subscribe_conversion_value)
            Reporting_Date.append(reporting_date)

        except UnicodeEncodeError as e:
            print(e)
        
        # 如果結束時間大於現在時間，則立即停止迴圈
        if now < end_time:
            end_time = now
            stop_flag += 1        
        
    print('第{}次'.format(count)) 
    time.sleep(60)
    
    if count%4 == 0:
        time.sleep(300)
    
    # 當開始時間和結束時間相等時，立即結束程式
    if start_time == end_time:
        break
    
# flag確立program完整結束
print('--- %s seconds ---' % (time.time() - run_time))

C:\ProgramData\Anaconda3\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of level might not be compatible.  Expect level_enum;  got <class 'set'>
  warnings.warn(message)
C:\ProgramData\Anaconda3\lib\site-packages\facebook_business\utils\api_utils.py:30: UserWarning: value of action_attribution_windows might not be compatible.  Expect list<action_attribution_windows_enum>;  got <class 'set'>
  warnings.warn(message)


2020-12-06
2020-12-12
第1次
2020-12-13
2020-12-19
第2次
2020-12-20
2020-12-26
第3次
2020-12-27
2021-01-02
第4次
2021-01-03
2021-01-08
第5次
--- 769.4446997642517 seconds ---


In [112]:
# 將資料以Dataaframe的方式儲存
raw_data = {'Campaign_Name': Campaign_Name,
         'Adset_Name': Adset_Name,
         'Objective': Objective,
         'Amount_Spent': Amount_Spent,
         'Amount_Install': Amount_Install,
         'Amount_Purchase': Amount_Purchase,
         'Purchase_Conversion_Value': Purchase_Conversion_Value,
         'Amount_Subscribe': Amount_Subscribe,
         'Subscribe_Conversion_Value': Subscribe_Conversion_Value,
         'Reporting_Date': Reporting_Date}
df = pd.DataFrame.from_dict(raw_data)
df

,Campaign_Name,Adset_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,Reporting_Date
0,武財神_APP下載_Android_20200723,武財神__Android_25-65+_興趣_道教_自動版位,APP_INSTALLS,259,31,,,,,20201206
1,武財神_APP下載_Android_20200723,武財神__Android_25-65+_類似受眾_安裝APP_自動版位,APP_INSTALLS,120,11,,,,,20201206
2,武財神_APP下載_Android_20200723,武財神__Android_25-65+_類似受眾_上香_自動版位,APP_INSTALLS,50,9,,,,,20201206
3,20200801_林恩如_AOS_下載_軟體,林恩如_Android_財經興趣,APP_INSTALLS,45,4,,,,,20201206
4,20200801_林恩如_AOS_下載_軟體,林恩如_Android_波段官網30D相似受眾,APP_INSTALLS,18,3,,,,,20201206
...,...,...,...,...,...,...,...,...,...,...
4209,20210105阿勳APP下載_Android_軟體,造訪陳重銘粉專類似受眾,APP_INSTALLS,190,4,,,,,20210108
4210,20210105阿勳APP下載_Android_軟體,造訪阿勳粉專類似受眾,APP_INSTALLS,216,3,,,,,20210108
4211,20210105阿勳APP下載_Android_軟體,造訪算利教官粉專類似受眾,APP_INSTALLS,407,5,,,,,20210108
4212,股市爆料同學會(非儲理財A)_testA_20210106,(理財A)股市爆料同學會_Android_20201217_測文案A,APP_INSTALLS,55,2,,,,,20210108


In [113]:
df = Expand_df(df)
df

,Serial_Number,Campaign_Name,Adset_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,Reporting_Date,Category
0,2020120600030801,武財神_APP下載_Android_20200723,武財神__Android_25-65+_興趣_道教_自動版位,APP_INSTALLS,259,31,,,,,20201206,FB
1,2020120600030802,武財神_APP下載_Android_20200723,武財神__Android_25-65+_類似受眾_安裝APP_自動版位,APP_INSTALLS,120,11,,,,,20201206,FB
2,2020120600030803,武財神_APP下載_Android_20200723,武財神__Android_25-65+_類似受眾_上香_自動版位,APP_INSTALLS,50,9,,,,,20201206,FB
3,2020120600030804,20200801_林恩如_AOS_下載_軟體,林恩如_Android_財經興趣,APP_INSTALLS,45,4,,,,,20201206,FB
4,2020120600030805,20200801_林恩如_AOS_下載_軟體,林恩如_Android_波段官網30D相似受眾,APP_INSTALLS,18,3,,,,,20201206,FB
...,...,...,...,...,...,...,...,...,...,...,...,...
4209,2021010800035010,20210105阿勳APP下載_Android_軟體,造訪陳重銘粉專類似受眾,APP_INSTALLS,190,4,,,,,20210108,FB
4210,2021010800035011,20210105阿勳APP下載_Android_軟體,造訪阿勳粉專類似受眾,APP_INSTALLS,216,3,,,,,20210108,FB
4211,2021010800035012,20210105阿勳APP下載_Android_軟體,造訪算利教官粉專類似受眾,APP_INSTALLS,407,5,,,,,20210108,FB
4212,2021010800035013,股市爆料同學會(非儲理財A)_testA_20210106,(理財A)股市爆料同學會_Android_20201217_測文案A,APP_INSTALLS,55,2,,,,,20210108,FB


In [114]:
df = None_to_Null(df)
df

,Serial_Number,Campaign_Name,Adset_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,Reporting_Date,Category
0,2020120600030801,武財神_APP下載_Android_20200723,武財神__Android_25-65+_興趣_道教_自動版位,APP_INSTALLS,259,31,None,None,None,None,20201206,FB
1,2020120600030802,武財神_APP下載_Android_20200723,武財神__Android_25-65+_類似受眾_安裝APP_自動版位,APP_INSTALLS,120,11,None,None,None,None,20201206,FB
2,2020120600030803,武財神_APP下載_Android_20200723,武財神__Android_25-65+_類似受眾_上香_自動版位,APP_INSTALLS,50,9,None,None,None,None,20201206,FB
3,2020120600030804,20200801_林恩如_AOS_下載_軟體,林恩如_Android_財經興趣,APP_INSTALLS,45,4,None,None,None,None,20201206,FB
4,2020120600030805,20200801_林恩如_AOS_下載_軟體,林恩如_Android_波段官網30D相似受眾,APP_INSTALLS,18,3,None,None,None,None,20201206,FB
...,...,...,...,...,...,...,...,...,...,...,...,...
4209,2021010800035010,20210105阿勳APP下載_Android_軟體,造訪陳重銘粉專類似受眾,APP_INSTALLS,190,4,None,None,None,None,20210108,FB
4210,2021010800035011,20210105阿勳APP下載_Android_軟體,造訪阿勳粉專類似受眾,APP_INSTALLS,216,3,None,None,None,None,20210108,FB
4211,2021010800035012,20210105阿勳APP下載_Android_軟體,造訪算利教官粉專類似受眾,APP_INSTALLS,407,5,None,None,None,None,20210108,FB
4212,2021010800035013,股市爆料同學會(非儲理財A)_testA_20210106,(理財A)股市爆料同學會_Android_20201217_測文案A,APP_INSTALLS,55,2,None,None,None,None,20210108,FB


In [115]:
Df_to_Csv(df)

In [116]:
df['Reporting_Date'] = df['Reporting_Date'].astype('str') #將資料型態轉為字串,但實際上不轉換寫入SQL時,SQL也會自動處理
InsertSQL(df, TableName)

## Send一封程式報告信

In [23]:
import traceback, sys
def error():
    print('執行發生錯誤!!')
    lastCallStack = traceback.extract_tb(sys.exc_info()[2])[0]    #取得Call Stack的第一筆資料
    lineNum = lastCallStack[1]                      #取得發生的行號
    log = 'problem occured in line {}. '.format(lineNum) + str(sys.exc_info()[1]).replace("\n", "").replace("'", '"')
    print(log)

In [25]:
while True:
    error()
    print('2323' + 123)


執行發生錯誤!!


IndexError: list index out of range

In [89]:
# 刪除資料
# TableName = 'FB_AdsData'
def DelSQL(TableName):
    str_query = """
        DELETE
        FROM [CMAPP].[dbo].[{}]
        WHERE Category LIKE '2021%'
        """
    print(str_query.format(TableName))
    DBdata_cr.execute(str_query.format(TableName))
    conn_DBdata.commit()   
    
DelSQL(TableName)



        DELETE
        FROM [CMAPP].[dbo].[FB_AdsData]
        WHERE Category LIKE '2021%'
        


## 出錯時手動回補資料

In [83]:
import numpy as np
import pandas as pd 

file = r'D:\FB_Ads_Marketing_Data_Analysis\20210105FBAdsTest.csv'
df = pd.read_csv(file)  #匯入資料
df['Reporting_Date'] = df['Reporting_Date'].astype('str') #將資料型態轉為字串,但實際上不轉換寫入SQL時,SQL也會自動處理
df

,Serial_Number,Campaign_Name,Adset_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,Category,Reporting_Date
0,2020071800013772,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,APP_INSTALLS,29,NaN,NaN,NaN,NaN,NaN,FB,20200718
1,2020071800013773,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,APP_INSTALLS,5,NaN,NaN,NaN,NaN,NaN,FB,20200718
2,2020071800013774,iCheck APP下載_20200324,iCheck APP下載_20190324_android (限動),APP_INSTALLS,23,NaN,NaN,NaN,NaN,NaN,FB,20200718
3,2020071800013775,iCheck APP下載_20200324,iCheck APP下載_20190324_iOS (限動),APP_INSTALLS,16,2.0,NaN,NaN,NaN,NaN,FB,20200718
4,2020071800013776,iCheck APP下載_20200324,iCheck APP下載_20190324_iOS (動態牆),APP_INSTALLS,147,11.0,NaN,NaN,NaN,NaN,FB,20200718
...,...,...,...,...,...,...,...,...,...,...,...,...
20950,2021010500034722,算利教官_購買_早鳥活動,基本面受眾_廣告圖片_圖片型LP,CONVERSIONS,370,NaN,18.0,7182.0,NaN,NaN,FB,20210105
20951,2021010500034723,算利教官_購買_早鳥活動,官方受眾_廣告圖片_圖片型LP,CONVERSIONS,426,NaN,6.0,2394.0,NaN,NaN,FB,20210105
20952,2021010500034724,算利教官_購買_早鳥活動,算利教官受眾_廣告圖片_動圖型LP,CONVERSIONS,405,NaN,21.0,16957.0,NaN,NaN,FB,20210105
20953,2021010500034725,算利教官_購買_早鳥活動,基本面受眾_廣告圖片_動圖型LP,CONVERSIONS,426,NaN,15.0,14563.0,NaN,NaN,FB,20210105


In [85]:

df = df[['Serial_Number', 'Campaign_Name', 'Adset_Name', 'Objective', 'Amount_Spent', 'Amount_Install', 'Amount_Purchase', 'Purchase_Conversion_Value', 'Amount_Subscribe', 'Subscribe_Conversion_Value', 'Reporting_Date', 'Category']]
df 

,Serial_Number,Campaign_Name,Adset_Name,Objective,Amount_Spent,Amount_Install,Amount_Purchase,Purchase_Conversion_Value,Amount_Subscribe,Subscribe_Conversion_Value,Reporting_Date,Category
0,2020071800013772,期貨電子盤_下載_IOS(舊),興趣期貨受眾_IOS,APP_INSTALLS,29,NaN,NaN,NaN,NaN,NaN,20200718,FB
1,2020071800013773,期貨電子盤_下載_IOS(舊),期貨電子盤類似受眾_IOS,APP_INSTALLS,5,NaN,NaN,NaN,NaN,NaN,20200718,FB
2,2020071800013774,iCheck APP下載_20200324,iCheck APP下載_20190324_android (限動),APP_INSTALLS,23,NaN,NaN,NaN,NaN,NaN,20200718,FB
3,2020071800013775,iCheck APP下載_20200324,iCheck APP下載_20190324_iOS (限動),APP_INSTALLS,16,2.0,NaN,NaN,NaN,NaN,20200718,FB
4,2020071800013776,iCheck APP下載_20200324,iCheck APP下載_20190324_iOS (動態牆),APP_INSTALLS,147,11.0,NaN,NaN,NaN,NaN,20200718,FB
...,...,...,...,...,...,...,...,...,...,...,...,...
20950,2021010500034722,算利教官_購買_早鳥活動,基本面受眾_廣告圖片_圖片型LP,CONVERSIONS,370,NaN,18.0,7182.0,NaN,NaN,20210105,FB
20951,2021010500034723,算利教官_購買_早鳥活動,官方受眾_廣告圖片_圖片型LP,CONVERSIONS,426,NaN,6.0,2394.0,NaN,NaN,20210105,FB
20952,2021010500034724,算利教官_購買_早鳥活動,算利教官受眾_廣告圖片_動圖型LP,CONVERSIONS,405,NaN,21.0,16957.0,NaN,NaN,20210105,FB
20953,2021010500034725,算利教官_購買_早鳥活動,基本面受眾_廣告圖片_動圖型LP,CONVERSIONS,426,NaN,15.0,14563.0,NaN,NaN,20210105,FB
